In [52]:
# Import Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [53]:
# Function to remove formatting from gatherer
def extract_name(card, card_long):
    length = len(card_long)
    good_string = card_long[length-len(card):-1]
    good_string += card_long[length-1]
    return(good_string) 

In [54]:
# Create browser to navigate oracle
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://gatherer.wizards.com/Pages/Default.aspx'
browser.visit(url)

In [55]:
# Here will be the list with all cards which will be converted from a csv.
# Temporarily, these cards are chosen so each mana symbol is represented at least once.
cards = ["Arcum's Astrolabe", "Jund Hackblade", "Ponder", "Stonecoil Serpent",
         "Beseech the Queen", "Thought-Knot Seer", "Apostle's Blessing", "Rakdos Carnarium"]

In [56]:
# This is where the analysis will be run

#cards = ["Goblin Ringleader"]

# Empty list for mismatched links is crafted
bad_link = []

# Empty lists are initialized.
# Mana is saved as [gen]eric, [snow], [c]olorless, [v]ariable, (two_b)rid, (phy)rexian, and
#    [w]hite, bl[u]e, [b]lack, [r]ed, and [g]reen (hybrid, phyrexian, and 2brid mana are counted twice).
# Converted Mana cost (cmc) is the total mana needed to cast (this will be less than total of columns if hybrid mana)
mana_gen = []
mana_snow = []
mana_c = []
mana_v = []
mana_two_b = []
mana_phy = []
mana_w = []
mana_u = []
mana_b = []
mana_r = []
mana_g = []
cmc = []

# For loop loops through list of cards
for card in cards:
    
    # Initialize all mana values which may be duplicated at 0
    gen = 0
    snow = 0
    c = 0
    v = 0
    two_b = 0
    phy = 0
    w = 0
    u = 0
    b = 0
    r = 0
    g = 0
    
    # Navigates browser to card page.  Note that this picks the first option in the dropdown when a card name is entered.
    # It is possible that the wrong card will be grabbed, so this is verified below.
    browser.fill("ctl00$ctl00$MainContent$Content$SearchControls$CardSearchBoxParent$CardSearchBox", card)
    time.sleep(1)
    browser.links.find_by_partial_text(card).click()
    
    # Soup object is crafted at card page so html can be read
    html = browser.html
    hot_soup = BeautifulSoup(html, "html.parser")
    
    # Check to see if card navigation was successful.  Name must be extracted from formatting to check strings are equal.
    # If strings are not equal, the entry is entered into a list.  Otherwise, the program continues.
    card_long = hot_soup.find("td", class_="rightCol").find("div", class_="value").text
    card_name = extract_name(card, card_long)
    if card != card_name:
        bad_link.append([card,card_name])
    
    
    # Card name is correct,so data is scraped
    else:
        
        # Check for a mana cost, if one does not exist, skip ahead
        try:
            mana_row = hot_soup.find("div", class_="row manaRow").find_all("img")
            
            # Loop through all mana symbols in the mana row, in each case the alt text will be used
            for cost in mana_row:
                
                # Attempt to turn symbol into a number, only possible if it is generic
                try:
                    gen = int(cost["alt"])
                
                # The symbol is not generic mana
                except:
                    
                    # These three qualities are unique, and thus mutually exclusive
                    if "Snow" == cost["alt"]:
                        snow += 1
                    elif "Colorless" == cost["alt"]:
                        c += 1
                    elif "Variable Colorless" == cost["alt"]:
                        v += 1
                    
                    # The following mana types may appear in combination in the same symbol, thus are checked if the string is
                    #    in the alt text.  The intent is that mana which may be paid for in multiple ways is counted as such.
                    else:
                        if "Two or " in cost["alt"]:
                            two_b += 1
                        if "Phyrexian" in cost["alt"]:
                            phy += 1
                        if "White" in cost["alt"]:
                            w += 1
                        if "Blue" in cost["alt"]:
                            u += 1
                        if "Black" in cost["alt"]:
                            b += 1
                        if "Red" in cost["alt"]:
                            r += 1
                        if "Green" in cost["alt"]:
                            g += 1    

        
        # If no mana cost, return all mana entries as None ("None" is different from "0", see Mountain and Ornithopter)
        except:
            gen = None
            snow = None
            c = None
            v = None
            two_b = None
            phy = None
            w = None
            u = None
            b = None
            r = None
            g = None

        
        
        # Check for a converted mana cost, if one does not exist, skip ahead
        try:
            ugly_number = hot_soup.find_all("div", class_="row")[2].find("div", class_="value").text
            cmc = int(ugly_number[len(ugly_number) - 1])
        except:
            cmc = None
        
            # Scrape the converted mana cost
        
        # Scrape the typeline
        
        # Check for a text box, if one does not exist, skip ahead
        
            # Scrape the text box
        
        # Check for power and toughness, if they do not exist, skip ahead
        
            # Scrape power and toughness
    
    # Navigates back to search page
    print(f"{card} has cmc of {cmc}.")
    browser.back()

Arcum's Astrolabe has cmc of 1.
Jund Hackblade has cmc of 2.
Ponder has cmc of 1.
Stonecoil Serpent has cmc of 0.
Beseech the Queen has cmc of 6.
Thought-Knot Seer has cmc of 4.
Apostle's Blessing has cmc of 2.
Rakdos Carnarium has cmc of None.
